In [ ]:
import pyscf
import pyscf.tools
import sys
sys.path.append('../../../OrbitalPartitioning/orbitalpartitioning')
import orbitalpartitioning
from orbitalpartitioning import *

ModuleNotFoundError: No module named 'pyscf'

In [1]:
 molecule = '''
 C       4.6336867757    -1.2242295397     0.0000000000
 C       4.4255614400     0.1700917199     0.0000000000
 C       5.9160648559    -1.7742824642     0.0000000000
 C       5.5765818320     0.9841959654     0.0000000000
 C       7.0417938838    -0.9473143671     0.0000000000
 C       6.8604981940     0.4376683883     0.0000000000
 C       0.5927492072     0.4581981628     0.0000000000
 C       0.4451033014     1.8584342989     0.0000000000
 C       1.9041039939    -0.0505338987     0.0000000000
 C       1.5640941394     2.6887327507     0.0000000000
 C       3.0514687259     0.7633813258     0.0000000000
 C       2.8520613255     2.1570405955     0.0000000000
 C      -6.8604981940    -0.4376683883     0.0000000000
 C      -7.0417938838     0.9473143671     0.0000000000
 C      -5.5765818320    -0.9841959654     0.0000000000
 C      -5.9160648559     1.7742824642     0.0000000000
 C      -4.4255614400    -0.1700917199     0.0000000000
 C      -4.6336867757     1.2242295397     0.0000000000
 C      -3.0514687259    -0.7633813258     0.0000000000
 C      -2.8520613255    -2.1570405955     0.0000000000
 C      -1.5640941394    -2.6887327507     0.0000000000
 C      -0.4451033014    -1.8584342989     0.0000000000
 C      -0.5927492072    -0.4581981628     0.0000000000
 C      -1.9041039939     0.0505338987     0.0000000000
 H       3.7849879120    -1.9076345469     0.0000000000
 H       6.0325833442    -2.8604965477     0.0000000000
 H       5.4821758879     2.0696091478     0.0000000000
 H       8.0463585128    -1.3756739224     0.0000000000
 H      -0.5426199813     2.3168156796     0.0000000000
 H       2.0368540411    -1.1280906363     0.0000000000
 H       1.4298106521     3.7726953568     0.0000000000
 H       3.6983932318     2.8425398416     0.0000000000
 H      -8.0463585128     1.3756739224     0.0000000000
 H      -5.4821758879    -2.0696091478     0.0000000000
 H      -6.0325833442     2.8604965477     0.0000000000
 H      -3.7849879120     1.9076345469     0.0000000000
 H      -3.6983932318    -2.8425398416     0.0000000000
 H      -1.4298106521    -3.7726953568     0.0000000000
 H       0.5426199813    -2.3168156796     0.0000000000
 H      -2.0368540411     1.1280906363     0.0000000000
 H      -7.7260669052    -1.1042415323     0.0000000000
 H       7.7260669052     1.1042415323     0.0000000000
 '''
basis = "cc-pvdz"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = pymol.nelec[1]
nsing = pymol.nelec[0] - ndocc
nvirt = pymol.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

NameError: name 'pyscf' is not defined

In [ ]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3,4,5):
        if ao[2] in ("2p"):
                print(ao)
                frag1.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (6,7,8,9,10,11):
        if ao[2] in ("2p"):
                frag2.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (12,13,14,15,16,17):
        if ao[2] in ("2p"):
                frag3.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (18,19,20,21,22,23):
        if ao[2] in ("2p"):
                frag4.append(ao_idx)
                full.append(ao_idx)            


frags = [frag1, frag2, frag3, frag4]
print(frags)
print(len(frag1))
print(len(frag2))
print(len(frag3))
print(len(frag4))

In [ ]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])

In [ ]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

In [ ]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
# Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
# Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)

## Make Integrals

In [ ]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)